In [1]:
from src.data.dataset_functions import get_dataframei_onlymods
import pyspark.sql.functions as F

mod_dataframe = get_dataframei_onlymods(4) #Der Parameter steht für den Datensatz, der verwendet werden soll

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-95c40c8c-67b2-4339-9b46-c28899ce6fb1;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 71ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

<div style="color:#b64365">

* Ich brauche eine Liste mit unterschiedlichen Farben, da Mods recht mit einheitlichen Farben zensiert haben,
    könnte also sonst problematisch werden zu erkennen was zsm.gehört und was nicht 
 
* Ich muss die zwei Koordinaten eines Mods in eine Liste von einzelenen Pixeln umwandeln, damit jeder Pixel gezeichnet wird.
* Oder ich lasse nur den Rahmen und nicht das komplette Quadrat zeichnen, dann könnte es aber vllt. schwierig auf dem Canvas zu erkennen sein?
    -> ich beginge wahrscheinlich erstmal nur mit dem Rahmen und schaue wie gut man das ganze erkennen kann
</div>

<div style="color:#d4597e">

* Im ersten Schritt schmeiße ich erstmal alle Spalten raus die nicht benötigt werden
* Anschließend versuche ich die Liste mit vielen unterschiedlichen Farben zu generieren
* Und danach eine Liste mit Pixeln für die Mods zu erstellen die gezeichnet werden soll

</div>

In [3]:
only_needed_columns = mod_dataframe.drop('t')
only_needed_columns.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- x1: integer (nullable = true)
 |-- y1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- y2: integer (nullable = true)
 |-- pixel_color: string (nullable = true)



In [4]:
import random

get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
#Der Parameter gibt an wie viele Farben man bekommt
print(get_colors(5)) # sample return:  ['#8af5da', '#fbc08c', '#b741d0', '#e599f1', '#bbcb59', '#a2a6c0']

['#692304', '#0951ca', '#08069f', '#0314c1', '#a47ba4']


In [5]:
#Um für alle Einträge eine Farbe zu bekommen, brauch ich hier nochmal die Anzahl an gemachten Einträgen
amount_of_needed_colors = only_needed_columns.count()

In [ ]:
#Nun die Liste erstellen für die Pixel

<div style="color:#d4597e">

Jetzt probiere ich mich am Canvas, verwende dafür Gretas Code mit den Bots als Beispiel.

</div>

In [ ]:
from PIL import Image
from src.visualization.visuialization_functions import parse_pixel_color

#Umwandlung in Pandas DataFrame, sollte hier aus Effizienzsicht nicht schlimm sein, da es nur wenige Mods gibt
pandasMods = only_needed_columns.toPandas()
canvas = Image.new("RGB", (2000,2000), "WHITE")

#Schrittweise Pixel im Canvas setzen
for index, row in pandasMods.iterrows():
    canvas.putpixel((row['x'],row['y']), parse_pixel_color(row['pixel_color']))

canvas.show()
#canvas.save('../reports/figures/mods-censors-canvas.jpg')